# Втор парцијален испит по Основи на Вештачката Интелигенција - 1/2

**Датум**: 18.01.2023

## 1. Амазон
Ја разгледуваме задачата за играта Амазон, од аудиториските вежби.

### Прашања и задачи

Бидејќи дрвото на играта е мало, во решението од аудиториските се разгранува целото дрво. Сакаме да дознаеме колку вкупно состојби се разгрануваат при пребарувањето низ дрвото.

1.1. **(5 поени)** Без користење на калкулатор, дискутирајте на кој начин ќе процените колку состојби треба да се разгранат за да се истражи целото дрво на играта.
    

* Ако сакаме да го разграниме целото дрво, тогаш не треба да користиме алфа бета поткаструвањe, значи ќе ја користeме функцијата minimax во која не фигурираат алфа и бета.
    * Во функцијата ќе поставиме бројач кој ќе брои колку пати ќе се повикува таа функција. Бидејќи секоја состојба од дрвото се повикува еднаш, значи бројот на повикувања = бројот на состојби кои се разгрануваат

1.2. **(5 поени) (код)** Пресметајте колку точно состојби се разгрануваат за да се истражи целото дрво на играта?

In [1]:
# Funkciite gi izvadov od klasata Game, zatoa sto vo momentot bitno mi e samo da ja odredam vrednosta na counter 
# (ne sakam da ja igram samata igra i ne mi treba kodot za vizuelizacija)

In [9]:
from copy import deepcopy

In [10]:
scores = {'S': 1, 'P': -1}
N = 3

def minimax(node, player):
    global counter
    counter += 1
    winner = check_victory(node, 'S' if player == 'MAX' else 'P')
    if winner != 'keep_playing':
        return scores[winner], None
    best_value = 2 if player == 'MIN' else -2
    best_move = None
    for child, move in expand_state(node, player):
        other_player = 'MIN' if player == 'MAX' else 'MAX'
        result, _ = minimax(child, other_player)
        if player == 'MIN' and result < best_value:
            best_value = result
            best_move = move
        elif player == 'MAX' and result > best_value:
            best_value = result
            best_move = move
    return best_value, best_move

def find_queen(state, queen_symbol):
    for y in range(N):
        for x in range(N):
            if state[y][x] == queen_symbol:
                return x, y

def possible_moves(state, x, y):
    deltas = [
        (0, 1), (0, -1), (1, 0), (-1, 0),
        (1, 1), (1, -1), (-1, 1), (-1, -1)]
    for dx, dy in deltas:
        nx, ny = x + dx, y + dy
        while 0 <= nx < N and 0 <= ny < N:
            if state[ny][nx] == '·':
                yield nx, ny
            else:
                break
            nx += dx
            ny += dy

def possible_shots(state, x, y):
    shots = [
        (0, 1), (0, -1), (1, 0), (-1, 0),
        (1, 1), (1, -1), (-1, 1), (-1, -1)]
    for dx, dy in shots:
        nx, ny = x + dx, y + dy
        if 0 <= nx < N and 0 <= ny < N:
            if state[ny][nx] == '·':
                yield nx, ny

def expand_state(state, player):
    symbol = 'S' if player == 'MAX' else 'P'
    qx, qy = find_queen(state, symbol)
    for px_move, py_move in possible_moves(state, qx, qy):
        state_after_move = list([list(row) for row in state])
        state_after_move[py_move][px_move] = symbol
        state_after_move[qy][qx] = '·'
        for px_shot, py_shot in possible_moves(state_after_move, px_move, py_move):
            state_after_shot = deepcopy(state_after_move)
            state_after_shot[py_shot][px_shot] = 'x'
            state_after_shot = tuple([tuple(row) for row in state_after_shot])
            yield state_after_shot, [qx, qy, px_move, py_move, px_shot, py_shot]

def other_queen_symbol(queen_symbol):
    return 'P' if queen_symbol == 'S' else 'S'

def check_victory(state, queen_to_move__symbol):
    queen_to_move__symbol
    qx, qy = find_queen(state, queen_to_move__symbol)
    if list(possible_moves(state, qx, qy)) == []:
        return other_queen_symbol(queen_to_move__symbol)
    return 'keep_playing'

In [11]:
counter = 0
# primer ova e nasata pocetna sostojba, a na red e MAX igracot
state = [
    ['·', '·', 'P'],
    ['·', '·', '·'],
    ['S', '·', '·'],
]

In [12]:
%%time
minimax(state, 'MAX')

CPU times: user 11.6 s, sys: 0 ns, total: 11.6 s
Wall time: 11.6 s


(1, [0, 2, 0, 1, 1, 1])

In [13]:
counter

907652

1.3. **(5 поени)** Што е алфа-бета поткастрување? Зошто користиме алфа-бета поткастрување во оваа игра?

* Алфа-бета поткаструвањето е метода која се користи со цел да се минимизира бројот на состојби кои се испитуваат. Ова овозможува minimax алгоритмот да се извршува во пократко време, но притоа повторно секогаш да го дава точниот резултат (поткаструвањето ги елиминира гранките кои сигурно нема да се крајното решение).
* Алфа-бета поткаструвањето се користи во играта со цел компјутерот побрзо да може да го пресмета наредниот чекор што играчот треба да го направи. (поточно што побрзо да изврши minimax)

1.4. **(10 поени) (код)** Покажете дека користењето на постапката за алфа-бета поткастрување се исплати во оваа игра. На кој начин подобро ќе се покаже: (1) преку времето потребно за извршување на алгоритмите или (2) преку броење на разгранетите јазли?

In [17]:
scores = {'S': 1, 'P': -1}
N = 3

def minimax_alpha_beta(node, player, alpha=-2, beta=2):
    global counter
    counter += 1
    winner = check_victory(node, 'S' if player == 'MAX' else 'P')
    if winner != 'keep_playing':
        return scores[winner], None
    best_value = 2 if player == 'MIN' else -2
    best_move = None
    for child, move in expand_state(node, player):
        other_player = 'MIN' if player == 'MAX' else 'MAX'
        result, _ = minimax_alpha_beta(child, other_player, alpha, beta)
        if player == 'MIN':
            if result <= alpha:
                return result, best_move
            if result < beta:
                beta = result
            if result < best_value:
                best_value = result
                best_move = move
        elif player == 'MAX':
            if result >= beta:
                return result, best_move
            if result > alpha:
                alpha = result
            if result > best_value:
                best_value = result
                best_move = move
    return best_value, best_move

# podolnite funkcii se istite kako vo kodot od zadaca 1.2
def find_queen(state, queen_symbol):
    for y in range(N):
        for x in range(N):
            if state[y][x] == queen_symbol:
                return x, y

def possible_moves(state, x, y):
    deltas = [
        (0, 1), (0, -1), (1, 0), (-1, 0),
        (1, 1), (1, -1), (-1, 1), (-1, -1)]
    for dx, dy in deltas:
        nx, ny = x + dx, y + dy
        while 0 <= nx < N and 0 <= ny < N:
            if state[ny][nx] == '·':
                yield nx, ny
            else:
                break
            nx += dx
            ny += dy

def possible_shots(state, x, y):
    shots = [
        (0, 1), (0, -1), (1, 0), (-1, 0),
        (1, 1), (1, -1), (-1, 1), (-1, -1)]
    for dx, dy in shots:
        nx, ny = x + dx, y + dy
        if 0 <= nx < N and 0 <= ny < N:
            if state[ny][nx] == '·':
                yield nx, ny

def expand_state(state, player):
    symbol = 'S' if player == 'MAX' else 'P'
    qx, qy = find_queen(state, symbol)
    for px_move, py_move in possible_moves(state, qx, qy):
        state_after_move = list([list(row) for row in state])
        state_after_move[py_move][px_move] = symbol
        state_after_move[qy][qx] = '·'
        for px_shot, py_shot in possible_moves(state_after_move, px_move, py_move):
            state_after_shot = deepcopy(state_after_move)
            state_after_shot[py_shot][px_shot] = 'x'
            state_after_shot = tuple([tuple(row) for row in state_after_shot])
            yield state_after_shot, [qx, qy, px_move, py_move, px_shot, py_shot]

def other_queen_symbol(queen_symbol):
    return 'P' if queen_symbol == 'S' else 'S'

def check_victory(state, queen_to_move__symbol):
    queen_to_move__symbol
    qx, qy = find_queen(state, queen_to_move__symbol)
    if list(possible_moves(state, qx, qy)) == []:
        return other_queen_symbol(queen_to_move__symbol)
    return 'keep_playing'

In [18]:
counter = 0
# primer ova e nasata pocetna sostojba, a na red e MAX igracot
state = [
    ['·', '·', 'P'],
    ['·', '·', '·'],
    ['S', '·', '·'],
]

In [19]:
%%time
minimax_alpha_beta(state, 'MAX')

CPU times: user 147 ms, sys: 0 ns, total: 147 ms
Wall time: 146 ms


(1, [0, 2, 0, 1, 1, 1])

In [20]:
counter

11147

* и преку времето и преку бројачот, очигледна е разликата што алфа бета поткаструвањето ја прави. Бројачот од околу 900.000 сега е околу 11.000, а времето од 11.5 секунди сега е од редот на милисекунди.

Целото дрво брзо се истражува на компјутер од 2022 година. Во продолжение разгледуваме случај кога овој алгоритам се поставува на компјутер кој нема доволно пресметковна моќ и мемориски ресурси за да го пребара целото дрво во логично време за еден потег. Ова е случај кога таблата наместо 3х3 ќе биде 6х6. Стандардната постапка во овој случај е да се ограничи длабочината до која пребарува алгоритамот, па тогаш да се проценува исходот од играта преку функција за евристика.

1.5. **(20 поени) (код)** Предложете функција за проценка на победникот (евристика) за оваа игра. Имплементирајте ја во код. Функцијата може да се напише без да ја тестирате со готовата игра од аудиториските вежби. Важно е да се објасни секоја логичка целина од кодот на функцијата која ќе ја напишете.

In [21]:
# Nekoi funckii go sodrzat zborot evolved bidejki koga go pisuvav kodot za natprevarot, prvo imav polosi verzii od ovie funkcii.
# Ne sakam da gi smenam iminjata bidejki nekade mozi ke zaboravam da smenam i se javi problem.

# ispituva dali kralicata moze vo eden poteg da stigni do goal_position
# ovaa funkcija e potrebna za ponatamu da se iskoristi vo how_many_squares_in_one_move_evolved
def can_reach_directly_evolved(goal_position, queen_position, state):
    gx, gy = goal_position
    qx, qy = queen_position
    if qy == gy: # ako kralicite se naogjaat vo ista kolona
        minx, maxx = (qx, gx) if qx < gx else (gx, qx)
        for x in range(minx+1, maxx):
            if state[qy][x] != '·': # ako izmegu dvete kralici ima precka
                return False
        return True # ako izmegu dvete kralici nema precki
    
    if qx == gx: # ako kralicite se naogjaat na ista redica
        miny, maxy = (qy, gy) if qy < gy else (gy, qy)
        for y in range(miny+1, maxy):
            if state[y][qx] != '·':
                return False
        return True
    
    if qx-qy == gx-gy: # na glavna dijagonala
        minx, maxx, miny = (qx, gx, qy) if qx < gx else (gx, qx, gy)
        for i in range(1, maxx-minx):
            if state[miny+i][minx+i] != '·':
                return False
        return True
    
    if qx+qy == gx+gy: # na sporedna dijagonala
        minx, maxx, miny = (qx, gx, qy) if qx < gx else (gx, qx, gy)
        for i in range(1, maxx-minx):
            if state[miny-i][minx+i] != '·':
                return False
        return True
    
    return False
   
# do kolku polinja kralicata moze da stigne vo eden poteg
def how_many_squares_in_one_move_evolved(state, queen_position):
    how_many = 0
    for row_index, row in enumerate(state):
        for col_index, element in enumerate(row):
            if element == '·':
                if can_reach_directly_evolved((col_index, row_index), queen_position, state):
                    how_many += 1
    return how_many
            

# ispituva koja kralica ima poveke mobilnost (moze vo eden poteg da stigne do poveke polinja).
# na izlez se vraka odreden score koj e megu -1 i 1 (ako e negativen broj, MIN kralicata ima
# prednost. Ako e pozitiven, MAX kralicata e vo prednost
def mobility(state):
    s_position = find_queen(state, 'S')
    p_position = find_queen(state, 'P')
    s_mobility = how_many_squares_in_one_move_evolved(state, s_position)
    p_mobility = how_many_squares_in_one_move_evolved(state, p_position)
    return (s_mobility - p_mobility) / 19 # najmnogu ima do 19 polinja mobilnost
                                          # toa e ako kralicata se naogja vo centarot na 6x6 tabla

def chebyshev(pos_1, pos_2):
    x1, y1 = pos_1
    x2, y2 = pos_2
    return max(abs(x1-x2), abs(y1-y2))


# Ispituva koja kralica e poblisku do odredeno pole t.e. go *poseduva* toa pole
# Ne e precizno bidejki ne zema vo predvid dali ima iksovci izmegu kralicata i odredenoto pole
def who_owns_square(state, square_position):
    s_position = find_queen(state, 'S')
    p_position = find_queen(state, 'P')
    s_dist = chebyshev(s_position, square_position)
    p_dist = chebyshev(p_position, square_position)
    if s_dist == p_dist:
        return 0
    return 1 if s_dist < p_dist else -1


# Kazuva koja kralica e vo prednost, koga se raboti za brojot na polinja koi tie gi poseduvaat
# Kralicata koja poseduva poveke polinja e vo prednost
def territory(state):
    score = 0
    how_many_squares = 0
    for row_index, row in enumerate(state):
        for col_index, element in enumerate(row):
            if element == '·':
                how_many_squares += 1 # how_many_squares na krajot ke pokazuva kolku vkupno prazni polinja ima na tablata
                score += who_owns_square(state, (col_index, row_index))
    return score / how_many_squares


# Kazuva kolku polinja direktno okolu kralicata se slobodni, podeleno so 8
# (bidejki maksimum 8 polinja moze da se slobodni okolu kralicata
def freedom_queen(state, symbol):
    N = len(state)
    how_many = 0
    x, y = find_queen(state, symbol)
    deltas = [
        (0, 1), (0, -1), (1, 0), (-1, 0),
        (1, 1), (1, -1), (-1, 1), (-1, -1)
    ]
    for dx, dy in deltas:
        nx, ny = x + dx, y + dy
        if 0 <= nx < N and 0 <= ny < N and state[ny][nx] == '·':
            how_many += 1
            nx += dx
            ny += dy
    return how_many/8


# Dava score za koja kralica e poslobodna
# Poslobodna e onaa kralica koja ima poveke polinja direktno okolu nea
def freedom(state):
    freedom_s = freedom_queen(state, 'S')
    freedom_p = freedom_queen(state, 'P')
    return freedom_s - freedom_p


# Konecnata evaluation function. Broevite se zemeni so testiranje.
def evaluate_state(state, symbol):
    return (3*mobility(state) + territory(state) + freedom(state)) / 5